<a href="https://colab.research.google.com/github/zeeeeeeeeun/bigdata-dacon/blob/main/logloss_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import random
from tqdm import tqdm
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, roc_auc_score,precision_score,recall_score, roc_curve, classification_report,precision_recall_curve
from sklearn.preprocessing import StandardScaler,RobustScaler,MinMaxScaler, Binarizer, OneHotEncoder, LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV, KFold

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
from google.colab import drive
mp = '/gdrive'
drive.mount(mp, force_remount=True)
path = mp + '/My Drive/Colab Notebooks/motion_cla/'

Mounted at /gdrive


In [3]:
X = pd.read_csv(path + "train_features.csv")
X

,id,time,acc_x,acc_y,acc_z,gy_x,gy_y,gy_z
0,0,0,1.206087,-0.179371,-0.148447,-0.591608,-30.549010,-31.676112
1,0,1,1.287696,-0.198974,-0.182444,0.303100,-39.139103,-24.927216
2,0,2,1.304609,-0.195114,-0.253382,-3.617278,-44.122565,-25.019629
3,0,3,1.293095,-0.230366,-0.215210,2.712986,-53.597843,-27.454013
4,0,4,1.300887,-0.187757,-0.222523,4.286707,-57.906561,-27.961234
...,...,...,...,...,...,...,...,...
1874995,3124,595,-0.712530,-0.658357,0.293707,-29.367857,-104.013664,-76.290437
1874996,3124,596,-0.683037,-0.658466,0.329223,-30.149089,-101.796809,-76.625087
1874997,3124,597,-0.664730,-0.666625,0.364114,-27.873095,-98.776072,-79.365125
1874998,3124,598,-0.630534,-0.682565,0.373696,-23.636550,-99.139495,-80.259478


In [4]:
y = pd.read_csv(path + "train_labels.csv")
y = y['label']
y

0       37
1       26
2        3
3       26
4       26
        ..
3120    26
3121    26
3122    15
3123    26
3124     2
Name: label, Length: 3125, dtype: int64

In [5]:
features = ['id', 'acc_x', 'acc_y', 'acc_z', 'gy_x', 'gy_y', 'gy_z']
X = X[features].groupby('id').agg(['max', 'min', 'mean','median', 'var'])
X

acc_x                      ...      gy_z                         
           max       min      mean  ...      mean     median           var
id                                  ...                                   
0     1.344268  0.591940  0.931329  ...  1.182107   1.913286    638.834979
1     1.234020 -2.156208 -0.766580  ...  1.393294  -0.655819   5707.098884
2     1.219836 -1.142847  0.039836  ...  3.053291   3.568888    193.775778
3    -0.622250 -1.417751 -0.887702  ... -5.869898  -3.853078    559.187841
4     0.599720 -2.429109 -0.659018  ...  4.453382  -0.750283   2129.668017
...        ...       ...       ...  ...       ...        ...           ...
3120  0.390798 -1.624711 -0.300454  ... -0.054026  -2.190464    620.698354
3121 -0.446650 -1.575455 -0.974298  ... -2.792238  -2.073951    163.493653
3122  0.744666 -2.578974 -1.114246  ... -1.722830  -7.884967  17401.991847
3123  0.915846 -0.929133 -0.111333  ... -5.930252 -13.745464   5075.586383
3124  0.538809 -1.013813 -0.434048  ... -3.246825   1.583578   2437.507157

[3125 rows x 30 columns]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=121, shuffle=True)

In [7]:
model = RandomForestClassifier(n_jobs=-1, random_state=0)

In [8]:
my_hyper_param = {  "n_estimators"     :[300] ,
                    "max_depth"        :[9],
                    "min_samples_leaf" :[1],
                    "random_state"     :[121,]
                 }

In [9]:
from sklearn.metrics import log_loss, make_scorer
LogLoss = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

In [10]:
gcv_model = GridSearchCV(model, param_grid=my_hyper_param, scoring=LogLoss, refit=True, cv=5, verbose=0)

In [11]:
gcv_model.fit(X_train, y_train)
print("best_estimator_", gcv_model.best_estimator_)
print("best_params_",    gcv_model.best_params_)
print("best_score_" ,    gcv_model.best_score_)
y_pred = gcv_model.predict_proba(X_test).astype(np.float64)
print(y_pred)

best_estimator_ RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=9, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=300,
                       n_jobs=-1, oob_score=False, random_state=121, verbose=0,
                       warm_start=False)
best_params_ {'max_depth': 9, 'min_samples_leaf': 1, 'n_estimators': 300, 'random_state': 121}
best_score_ -1.1977753590118134
[[4.22251905e-03 3.07188806e-04 3.11870612e-04 ... 4.84419126e-04
  1.66565929e-03 2.22965374e-03]
 [3.18837265e-03 3.29488038e-04 4.11204953e-04 ... 5.74581631e-04
  2.06002204e-04 2.68092962e-03]
 [4.65549978e-03 1.05587780e-02 2.92621379e-03 ... 4.24451347e-04
  2.95845573e-05 2.68029245e-02]
 ...
 [2.08967436e-